<a href="https://colab.research.google.com/github/lx-jdar/progra-concurrente/blob/development/tp3/go/tp3_golang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo práctico 1: GOLANG

Instalacion de golang en Colab

In [ ]:
!add-apt-repository ppa:longsleep/golang-backports -y
!apt update
!apt install golang-go
%env GOPATH=/root/go
!go version

Creación del Archivo GO

In [ ]:
%%writefile tp3-golang.go
package main

import (
	"fmt"
	"os"
	"regexp"
	"strings"
	"sync"
)

const (
	ASCII_START     = 64
	CANT_THREADS    = 2
	CHARS_BY_THREAD = 4
	INIT_VALUE      = 0
	WORKER_A        = "A"
	WORKER_B        = "B"
)

type CharPackage struct {
	threadID string
	offset   int
	chars    string
}

var password []int
var mtx sync.Mutex

func convertirAEntero(chnl chan CharPackage, wg *sync.WaitGroup) {
	defer wg.Done()

	data := <-chnl
	cadena := data.chars
	charSize := len(cadena)
	cycles := charSize / CHARS_BY_THREAD
	if charSize%CHARS_BY_THREAD > 0 {
		cycles++
	}
	for cycle := 0; cycle < cycles; cycle++ {
		startIdx := cycle * CHARS_BY_THREAD
		endIdx := startIdx + CHARS_BY_THREAD

		if endIdx > charSize {
			endIdx = charSize
		}
		fmt.Printf("Worker%s procesando %s\n", data.threadID, cadena[startIdx:endIdx])
		for idx := startIdx; idx < endIdx; idx++ {
			mtx.Lock()
			password[idx+data.offset] = int(data.chars[idx]) - ASCII_START
			mtx.Unlock()
		}
	}
}

func displayError() {
	fmt.Println("Use: go run tp3-golang.go WordToCypher")
	fmt.Println("[WordToCypher] debe ser al menos una letra del Abecedario A-Z")
	panic("Programa tp3-golang.go mal REALIZADO!")
}

func main() {

	if len(os.Args) < 2 {
		displayError()
	}
	cadena := strings.ToUpper(os.Args[1])
	matched, _ := regexp.MatchString(`^[A-Z]+$`, cadena)
	if !matched {
		displayError()
	}

	password = make([]int, len(cadena))
	dataT1 := make(chan CharPackage)
	dataT2 := make(chan CharPackage)

	var wg sync.WaitGroup

	// creo los threads que tratan los caracteres
	wg.Add(CANT_THREADS)
	go convertirAEntero(dataT1, &wg)
	go convertirAEntero(dataT2, &wg)

	endChars := len(cadena)
	cantChars := endChars / 2
	fmt.Println("######## Distribucion de cadenas ########")
	fmt.Printf("Worker%s: %s\n", WORKER_A, string(cadena[INIT_VALUE:cantChars]))
	fmt.Printf("Worker%s: %s\n\n", WORKER_B, string(cadena[cantChars:endChars]))
	dataT1 <- CharPackage{WORKER_A, INIT_VALUE, string(cadena[INIT_VALUE:cantChars])}
	dataT2 <- CharPackage{WORKER_B, cantChars, string(cadena[cantChars:endChars])}

	close(dataT1)
	close(dataT2)

	wg.Wait()
	fmt.Println("\nCadena Cifrada: ", cadena)
	fmt.Println("Cifrado: ", password)
}

Compilacion del Archivo GO

In [ ]:
!go build tp3-golang.go

Ejecución del Programa con diferentes ejemplos

In [ ]:
!./tp3-golang ARBoLito

######## Distribucion de cadenas ########
WorkerA: ARBO
WorkerB: LITO

WorkerA procesando ARBO
WorkerB procesando LITO

Cadena Cifrada:  ARBOLITO
Cifrado:  [1 18 2 15 12 9 20 15]


In [ ]:
!./tp3-golang ARBoLiyOstes

######## Distribucion de cadenas ########
WorkerA: ARBOLI
WorkerB: YOSTES

WorkerA procesando ARBO
WorkerA procesando LI
WorkerB procesando YOST
WorkerB procesando ES

Cadena Cifrada:  ARBOLIYOSTES
Cifrado:  [1 18 2 15 12 9 25 15 19 20 5 19]


In [ ]:
!./tp3-golang ARBoLiTOEMAUSPEREdfdfesrA

######## Distribucion de cadenas ########
WorkerA: ARBOLITOEMAU
WorkerB: SPEREDFDFESRA

WorkerA procesando ARBO
WorkerA procesando LITO
WorkerA procesando EMAU
WorkerB procesando SPER
WorkerB procesando EDFD
WorkerB procesando FESR
WorkerB procesando A

Cadena Cifrada:  ARBOLITOEMAUSPEREDFDFESRA
Cifrado:  [1 18 2 15 12 9 20 15 5 13 1 21 19 16 5 18 5 4 6 4 6 5 19 18 1]


# Comentarios

> Golang fue desarrollado en Google, con lo cual ah sacado lo mejor de diversos lenguajes tales como C/C++, Java y Python. De a poco se va utilizando en el desarrollo de diversos programas. En cuanto a la concurrencia, está altamente desarrollado para gestionar diversas tareas con distintos mecanismo de comunicaciones. Usa principalmente lo que se conoce como channels que es similar a pipes en C/C++, pero también incorpora semaphore mtx, etc. Su sintaxis puede ser dificil de entender al principio, pero es completamente amigable para el desarrollo de cualquier programa.

# Conclusión

> Golang requiere de mayor documentación para que sea mejor utilizado y entendido en la comunidad. Pero es muy versatil para poder realizar principalmente en tareas concurrente debido a su sencilla sintaxis e incorporación de diversos lenguajes como eje de programción.

